In [1]:
import pandas as pd
df = pd.read_csv('datasets_4123_6408_framingham.csv')
df.head()

C:\Users\Verma's\AppData\Local\Temp\ipykernel_15528\1247821241.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


DATA PREPROCESSING

In [3]:
df.shape #to find no. of records

(4240, 16)

In [6]:
df.drop('education',axis=1,inplace=True)#axis 1 for column and inplace true for hamesha k liye rkhne k liye

In [7]:
# to fill missing valuse and delete use less data
df.isnull().sum() # this tells missing data in each column

male                 0
age                  0
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [12]:
#fill the binary values with mode funciton means jo jyda repeat ho rhi ho valuse
#fill the continuous columns wiht meadian values 
# Define the binary columns
bin_cols = ["male", "currentSmoker", "prevalentStroke", "prevalentHyp", "diabetes"]

# Fill missing values for binary features with the most frequent value (mode)
for col in bin_cols:
    mode_val = df[col].mode()[0]
    df.fillna({col: mode_val}, inplace=True)

# Check if there are any remaining missing values
df.isnull().sum()

male                 0
age                  0
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [15]:
# Import necessary libraries
import numpy as np

# Fill missing values for numeric features
numeric_cols = ["cigsPerDay", "BPMeds", "totChol", "BMI", "heartRate", "glucose"]
for col in numeric_cols:
    median_val = df[col].median()
    df.fillna({col:median_val}, inplace=True)


# Check if there are any remaining missing values
df.isnull().sum() #see now no missing value

male               0
age                0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [16]:
#balanced dataset 
df['TenYearCHD'].value_counts()#as here 0 have more values then 1 so we have to balance the dataset to make a good model
#we have three ways oversampling undersampling smote

TenYearCHD
0    3596
1     644
Name: count, dtype: int64

In [17]:
#balancing the data
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['TenYearCHD'] == 0] #extrating 0 values
df_minority = df[df['TenYearCHD'] == 1] #extrating 1 values

#here we are doing oversampling
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # Sample with replacement
                                 n_samples=len(df_majority),    # To match majority class
                                 random_state=42) # Reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [18]:
df_balanced['TenYearCHD'].value_counts()

TenYearCHD
0    3596
1    3596
Name: count, dtype: int64

In [20]:
#now splitting the data in train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features (X) and target variable (y)
X = df_balanced.drop(columns=['TenYearCHD']) # only input values
y = df_balanced['TenYearCHD'] # output values

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#0.2 means x 20% data utha k x_test ko de dega and same with y

In [22]:
X_train
#now here in tochol etc columns m some values are very large some values are small so we'll use scaler to bring these values to 0-1 
#because vrna model large value ko more importance deta h
#two techniques minmax scaler and standard scaler

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
1905,0,64,0,0.0,0.0,0,1,0,229.0,145.0,85.0,29.67,70.0,74.0
2075,0,37,1,20.0,0.0,0,0,0,166.0,112.0,73.5,21.64,75.0,93.0
1128,0,63,1,10.0,0.0,0,1,0,236.0,189.0,103.0,27.91,60.0,74.0
1782,0,65,0,0.0,0.0,0,1,0,245.0,171.0,89.0,23.07,82.0,93.0
241,1,65,1,15.0,0.0,0,1,0,219.0,148.0,90.0,29.35,77.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,1,63,1,20.0,0.0,0,1,0,269.0,180.0,101.0,24.42,72.0,84.0
485,1,54,1,40.0,0.0,0,0,0,230.0,145.0,90.0,25.72,75.0,85.0
4232,1,68,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0
952,1,66,1,30.0,0.0,0,0,1,234.0,114.5,62.5,28.62,75.0,216.0


In [24]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler to training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled#numpy array

array([[-0.94172615,  1.4582083 , -1.02624378, ...,  0.840088  ,
        -0.50731448, -0.34072887],
       [-0.94172615, -1.66694628,  0.97442734, ..., -1.050363  ,
        -0.0898974 ,  0.25133934],
       [-0.94172615,  1.34246184,  0.97442734, ...,  0.42574258,
        -1.34214864, -0.34072887],
       ...,
       [ 1.06187982,  1.92119417, -1.02624378, ..., -0.6972277 ,
        -1.34214864, -0.18492145],
       [ 1.06187982,  1.68970124,  0.97442734, ...,  0.59289329,
        -0.0898974 ,  4.08420194],
       [ 1.06187982, -0.74097455,  0.97442734, ...,  1.0660946 ,
         0.32751968,  1.84057505]])

TRAIN

WE WILL BE TRAINING WITH 10 DIFFERENT MODELS AND DIFFERENT METRICS

In [44]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


# Define a list of classifiers
classifiers = [
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    XGBClassifier()
]

# Create a dictionary to store the results
results = {}

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1-Score', 'Precision', 'Recall'])


# Train and evaluate each classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{clf_name} Accuracy: {accuracy}")
    
    # Classification report
    print(f"Classification Report for {clf_name}:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    print(f"Confusion Matrix for {clf_name}:")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    f1_score = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']


RandomForestClassifier Accuracy: 0.9722029186935371
Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       735
           1       0.95      0.99      0.97       704

    accuracy                           0.97      1439
   macro avg       0.97      0.97      0.97      1439
weighted avg       0.97      0.97      0.97      1439

Confusion Matrix for RandomForestClassifier:
[[700  35]
 [  5 699]]


c:\Users\Verma's\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier Accuracy: 0.6719944405837387
Classification Report for AdaBoostClassifier:
              precision    recall  f1-score   support

           0       0.69      0.66      0.67       735
           1       0.66      0.68      0.67       704

    accuracy                           0.67      1439
   macro avg       0.67      0.67      0.67      1439
weighted avg       0.67      0.67      0.67      1439

Confusion Matrix for AdaBoostClassifier:
[[486 249]
 [223 481]]
GradientBoostingClassifier Accuracy: 0.7289784572619875
Classification Report for GradientBoostingClassifier:
              precision    recall  f1-score   support

           0       0.76      0.69      0.72       735
           1       0.70      0.77      0.74       704

    accuracy                           0.73      1439
   macro avg       0.73      0.73      0.73      1439
weighted avg       0.73      0.73      0.73      1439

Confusion Matrix for GradientBoostingClassifier:
[[508 227]
 [163 541]]
Logist

BEST MODEL

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the RandomForestClassifier
rf_classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Classifier Accuracy:", accuracy_rf)

# Classification report
print("Classification Report for Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

# Confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_rf))

Random Forest Classifier Accuracy: 0.9715079916608756
Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       735
           1       0.95      0.99      0.97       704

    accuracy                           0.97      1439
   macro avg       0.97      0.97      0.97      1439
weighted avg       0.97      0.97      0.97      1439

Confusion Matrix for Random Forest Classifier:
[[699  36]
 [  5 699]]


In [46]:
# test 1:
print("predcted class ",rf_classifier.predict(X_test_scaled[10].reshape(1,-1))[0])
print("actual class ", y_test.iloc[10])

predcted class  0
actual class  0


In [58]:
X_test_scaled[200].reshape(1,-1)

array([[ 1.06187982, -1.78269275,  0.97442734,  0.81185379, -0.20499115,
        -0.09734139, -0.79968646, -0.21580282,  0.02868136, -0.76044968,
         0.00286132, -0.48534651,  0.16055284,  0.56295418]])

In [47]:
# test 2:
print("predcted class ",rf_classifier.predict(X_test_scaled[200].reshape(1,-1))[0])
print("actual class ", y_test.iloc[200])

predcted class  1
actual class  1


In [48]:
#saving files because we have to use in website
#we will save two things one our model and one scaler that we use
#we will use pickel for that

In [52]:
import pickle
pickle.dump(rf_classifier,open("model/rf_classifier.pkl",'wb'))
pickle.dump(scaler,open("model/scaler.pkl",'wb'))
#wb => write binary mode rb=> read binary mode

In [53]:
# Load the RandomForestClassifier model
with open("model/rf_classifier.pkl", "rb") as file:
    rf_classifier = pickle.load(file)

# Load the scaler
with open("model/scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

MAKING PREDICTOR FUNCTION

In [54]:
#we'll make a funciton firstly it will encode yes no male female valuse to 0 and 1
#then convert the values to array using np.array
#then scale the values scaler.transform
#then predict the answer model_name.predict(scaled_features)
def predict(model, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose):
    # Encode categorical variables
    male_encoded = 1 if male.lower() == "male" else 0
    currentSmoker_encoded = 1 if currentSmoker.lower() == "yes" else 0
    BPMeds_encoded = 1 if BPMeds.lower() == "yes" else 0
    prevalentStroke_encoded = 1 if prevalentStroke.lower() == "yes" else 0
    prevalentHyp_encoded = 1 if prevalentHyp.lower() == "yes" else 0
    diabetes_encoded = 1 if diabetes.lower() == "yes" else 0
    
    # Prepare features array
    features = np.array([[male_encoded, age, currentSmoker_encoded, cigsPerDay, BPMeds_encoded, prevalentStroke_encoded, prevalentHyp_encoded, diabetes_encoded, totChol, sysBP, diaBP, BMI, heartRate, glucose]])
    
    # scalling
    scaled_features = scaler.transform(features)
    
    # predict by model
    result = model.predict(scaled_features)
    
    return result[0]

In [55]:
# test 1:
male = "female"
age = 56.00
currentSmoker = "yes"
cigsPerDay = 3.00
BPMeds = "no"
prevalentStroke = "no"
prevalentHyp = "yes"
diabetes = 'no'
totChol = 285.00
sysBP = 145.00
diaBP = 100.00
BMI = 30.14
heartRate = 80.00
glucose = 86.00


result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)

if result == 1:
    print("The Patient has Heart Diseas")
else: 
    print("The Patiennt has No Heart Deseas")

The Patiennt has No Heart Deseas


c:\Users\Verma's\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


SO NOW WE HAVE PREPROCESSED TRAIN TEST METRICS EVALUATE PREDICT THE DATA 
NOW MOVING TO MAKE GUI PART OF THE PROJECT

In [56]:
import sklearn

sklearn.__version__

'1.4.1.post1'